In [1]:
#import packages and libraries
!pip install bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library
!pip install scikit-learn==0.23.1
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



### Getting neighborhood information from Wikipedia

In [2]:
#Getting the html and parsing using beautiful soup
url ="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get(url).text
Toronto_soup = BeautifulSoup(html_data, "html5lib")
table_contents=[]
Zip_table = Toronto_soup.find('table')

#loop to extract the content
for item in Zip_table.find_all('td'):
    cell = {}
    if item.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = item.p.text[:3]
        cell['Borough'] = (item.span.text).split('(')[0]
        cell['Neighborhood'] = (((((item.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#convert the data structture into a Dataframe
df_Toronto = pd.DataFrame(table_contents)
df_Toronto['Borough']=df_Toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df_Toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Getting coordinates for each location

In [3]:
#get the latitude and longitude for each location
!pip install pgeocode
import pgeocode

In [4]:
locator=pgeocode.Nominatim('ca')
list_zip_codes=df_Toronto['PostalCode'].tolist()
latitude=[]
longitude=[]

for code in list_zip_codes:
    g=locator.query_postal_code(code)
    latitude.append(g.latitude)
    longitude.append(g.longitude)

df_Toronto["lat"]=latitude
df_Toronto["long"]=longitude
print(df_Toronto.shape)
df_Toronto.head()

(103, 5)


,PostalCode,Borough,Neighborhood,lat,long
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Queen's Park,Ontario Provincial Government,43.6641,-79.3889


### Let's plot this dataframe into a map

In [5]:
#Find if there is any missing or Nan value 
is_Nan = df_Toronto[df_Toronto.isnull().any(axis=1)]
print(is_Nan)

   PostalCode      Borough    Neighborhood  lat  long
76        M7R  Mississauga  Enclave of L4W  NaN   NaN


In [6]:
#Replace the missing values manually with the correct information
df_Toronto.loc[76,'lat']=43.63697
df_Toronto.loc[76,'long']=-79.6158
print([df_Toronto.loc[76,]])

[PostalCode                 M7R
Borough            Mississauga
Neighborhood    Enclave of L4W
lat                   43.63697
long                  -79.6158
Name: 76, dtype: object]


In [7]:
#Plot the map of locations
g_map=locator.query_postal_code('M5A') # Lat et Long of Downtown Toronto
lat_map=g_map.latitude
long_map=g_map.longitude

#let's plot the map of Toronto
map_Toronto = folium.Map(location=[lat_map, long_map],zoom_start=10)

for lat, lng, label in zip(df_Toronto['lat'], df_Toronto['long'], df_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)

map_Toronto

### We can limit our dataframe to the boroughs located in the city

In [8]:
df_Toronto_city= df_Toronto[df_Toronto['Borough'].str.contains('Toronto')]
print(df_Toronto_city.shape)
df_Toronto_city.head()

(39, 5)


,PostalCode,Borough,Neighborhood,lat,long
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
19,M4E,East Toronto,The Beaches,43.6784,-79.2941
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754


### Let's now plot the boroughs located in the city

In [9]:
#We plot the Neighborhood located in the city
map_Toronto_city = folium.Map(location=[lat_map, long_map],zoom_start=11)

for lat, lng, label in zip(df_Toronto_city['lat'], df_Toronto_city['long'], df_Toronto_city['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto_city)

map_Toronto_city

## Now let's explore the venues

In [10]:
# @hidden_cell
CLIENT_ID = 'QLM2DB1AUDR2J2XCV1OGFM0FMOKG2MBZHWBUJ3DTLE4AQVKB' # your Foursquare ID
CLIENT_SECRET = 'Z13OJUA5Z5QQ0JLRHLGZBLSNHFUARL2MHEUGUNH4R4DSCUGS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

### We create a function in order to retrieve the venues for the neighborhoods

In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Now we  get the venues for each neighborhood

In [12]:
neighborhood_names = df_Toronto_city['Neighborhood']
neighborhood_lats = df_Toronto_city['lat']
neighborhood_longs = df_Toronto_city['long']
toronto_venues = getNearbyVenues(neighborhood_names, neighborhood_lats, neighborhood_longs)

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [13]:
print(toronto_venues.shape)
toronto_venues.head()

(1436, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.6555,-79.3626,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
3,"Regent Park, Harbourfront",43.6555,-79.3626,Berkeley Church,43.655123,-79.365873,Event Space
4,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


Let's get the number of venues for each neighborhood

In [14]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,75,75,75,75,75,75
"Brockton, Parkdale Village, Exhibition Place",40,40,40,40,40,40
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",60,60,60,60,60,60
Central Bay Street,58,58,58,58,58,58
Christie,10,10,10,10,10,10
Church and Wellesley,69,69,69,69,69,69
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,18,18,18,18,18,18
Davisville North,7,7,7,7,7,7


Let's see the most frequent categories

In [15]:
#Lets see the most frequent categories
df_categories= toronto_venues.groupby('Venue Category').count()
df_categories.sort_values(by="Venue",ascending=False).head(15)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Coffee Shop,149,149,149,149,149,149
Café,81,81,81,81,81,81
Sandwich Place,63,63,63,63,63,63
Restaurant,44,44,44,44,44,44
Japanese Restaurant,41,41,41,41,41,41
Hotel,36,36,36,36,36,36
Italian Restaurant,35,35,35,35,35,35
Bank,33,33,33,33,33,33
Sushi Restaurant,32,32,32,32,32,32


### Let's apply the one hot encoding and grouped all the venues for each neighborhood

In [16]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000,0.000000,0.000000,0.000000,0.026667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.026667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.025,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.033333,0.000000,0.016667
3,Central Bay Street,0.000,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.017241,0.017241,0.017241,0.000000,0.000000,0.000000,0.000000
4,Christie,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.000,0.014493,0.014493,0.014493,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014493,0.000000,0.000000,0.014493
6,"Commerce Court, Victoria Hotel",0.000,0.000000,0.000000,0.020000,0.010000,0.000000,0.040000,0.000000,0.000000,...,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
7,Davisville,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000


### Let's now sort and display the top 10 venues for each neighborhood

In [17]:
#Let's create the function that sorts the venues for each neighborhood

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [18]:
#Now let's put this into a dataframe and display them
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Sandwich Place,Coffee Shop,Cocktail Bar,Hotel,Beer Bar,Café,Bakery,Japanese Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Coffee Shop,Sandwich Place,Japanese Restaurant,Café,Thrift / Vintage Store,Breakfast Spot,Arts & Crafts Store,Gift Shop,Mexican Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Italian Restaurant,Bar,New American Restaurant,Bakery,Burger Joint,Sandwich Place,Restaurant,Pizza Place,Park
3,Central Bay Street,Coffee Shop,Sandwich Place,Japanese Restaurant,Sushi Restaurant,Pizza Place,Italian Restaurant,Café,Middle Eastern Restaurant,Video Game Store,Bubble Tea Shop
4,Christie,Café,Grocery Store,Coffee Shop,Park,Athletics & Sports,Baby Store,Accessories Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


### Clusteing the neighborhoods

We will apply the K-means algorithm in order to cluster the neighborhoods. We are going to use 4 general clusters and we are going to use all the venues which are grouped in the toronto_grouped dataframe

In [19]:
kclusters = 4
toronto_cluster_df = toronto_grouped.drop('Neighborhood', 1)

#apply k-means algorithm

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_cluster_df)
kmeans.labels_[0:10]


array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

We are now merge the dataframes

In [20]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_city_merged = df_Toronto_city

toronto_city_merged = toronto_city_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_city_merged.head(10) 

,PostalCode,Borough,Neighborhood,lat,long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,2.0,Coffee Shop,Thai Restaurant,Electronics Store,Restaurant,Pub,Beer Store,Greek Restaurant,Breakfast Spot,Event Space,Thrift / Vintage Store
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,2.0,Coffee Shop,Clothing Store,Sandwich Place,Café,Hotel,Cosmetics Shop,Pizza Place,Bank,Middle Eastern Restaurant,Japanese Restaurant
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,2.0,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Gastropub,Seafood Restaurant,Restaurant,Clothing Store,Cosmetics Shop,Department Store
19,M4E,East Toronto,The Beaches,43.6784,-79.2941,2.0,Pub,Gastropub,Health Food Store,Pizza Place,Bakery,Cheese Shop,Accessories Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,2.0,Sandwich Place,Coffee Shop,Cocktail Bar,Hotel,Beer Bar,Café,Bakery,Japanese Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant
24,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.3860,2.0,Coffee Shop,Sandwich Place,Japanese Restaurant,Sushi Restaurant,Pizza Place,Italian Restaurant,Café,Middle Eastern Restaurant,Video Game Store,Bubble Tea Shop
25,M6G,Downtown Toronto,Christie,43.6683,-79.4205,2.0,Café,Grocery Store,Coffee Shop,Park,Athletics & Sports,Baby Store,Accessories Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833,2.0,Coffee Shop,Café,Sandwich Place,Hotel,Gym,Asian Restaurant,Sushi Restaurant,Restaurant,Deli / Bodega,Japanese Restaurant
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6655,-79.4378,2.0,Park,Bakery,Pharmacy,Bank,Middle Eastern Restaurant,Bus Line,Furniture / Home Store,Brazilian Restaurant,Liquor Store,Smoke Shop
35,M4J,East York/East Toronto,The Danforth East,43.6872,-79.3368,0.0,Rental Car Location,Convenience Store,Park,Intersection,Music Venue,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant


In [21]:
# @hidden_cell
print(toronto_city_merged.shape)
is_Nan = toronto_city_merged[toronto_city_merged.isnull().any(axis=1)]
print(is_Nan)
toronto_city_merged.drop([62], inplace=True)
toronto_city_merged.shape

(39, 16)
   PostalCode          Borough Neighborhood      lat     long  Cluster Labels  \
62        M5N  Central Toronto     Roselawn  43.7113 -79.4195             NaN   

   1st Most Common Venue 2nd Most Common Venue 3rd Most Common Venue  \
62                   NaN                   NaN                   NaN   

   4th Most Common Venue 5th Most Common Venue 6th Most Common Venue  \
62                   NaN                   NaN                   NaN   

   7th Most Common Venue 8th Most Common Venue 9th Most Common Venue  \
62                   NaN                   NaN                   NaN   

   10th Most Common Venue  
62                    NaN  


(38, 16)

### Let's now visualize the clusters

In [22]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_toronto_clusters = folium.Map(location=[lat_map, long_map],zoom_start=11)

#Set colors for each cluster
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_city_merged['lat'], toronto_city_merged['long'], toronto_city_merged['Neighborhood'], toronto_city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_toronto_clusters)
       
map_toronto_clusters

## Let's examine each Cluster

Cluster 1

In [24]:
toronto_city_merged.loc[toronto_city_merged['Cluster Labels'] == 0, toronto_city_merged.columns[[1] + list(range(5,toronto_city_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,East York/East Toronto,0.0,Rental Car Location,Convenience Store,Park,Intersection,Music Venue,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant
36,Downtown Toronto,0.0,Park,Harbor / Marina,Café,Music Venue,Accessories Store,Museum,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
61,Central Toronto,0.0,Photography Studio,Park,Lawyer,Museum,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant
67,Central Toronto,0.0,Food & Drink Shop,Breakfast Spot,Park,Department Store,Playground,Gym / Fitness Center,Dog Run,Museum,Mexican Restaurant,Middle Eastern Restaurant
73,Central Toronto,0.0,Park,Playground,Gym Pool,Garden,Accessories Store,Movie Theater,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
83,Central Toronto,0.0,Park,Gym,Grocery Store,Trail,Thai Restaurant,Accessories Store,Museum,Men's Store,Mexican Restaurant,Middle Eastern Restaurant
91,Downtown Toronto,0.0,Playground,Park,Grocery Store,Candy Store,Accessories Store,Museum,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant


Cluster 2

In [25]:
toronto_city_merged.loc[toronto_city_merged['Cluster Labels'] == 1, toronto_city_merged.columns[[1] + list(range(5,toronto_city_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,1.0,Trail,Accessories Store,Music Venue,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark


Cluster 3

In [26]:
toronto_city_merged.loc[toronto_city_merged['Cluster Labels'] == 2, toronto_city_merged.columns[[1] + list(range(5,toronto_city_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2.0,Coffee Shop,Thai Restaurant,Electronics Store,Restaurant,Pub,Beer Store,Greek Restaurant,Breakfast Spot,Event Space,Thrift / Vintage Store
9,Downtown Toronto,2.0,Coffee Shop,Clothing Store,Sandwich Place,Café,Hotel,Cosmetics Shop,Pizza Place,Bank,Middle Eastern Restaurant,Japanese Restaurant
15,Downtown Toronto,2.0,Coffee Shop,Cocktail Bar,Café,Italian Restaurant,Gastropub,Seafood Restaurant,Restaurant,Clothing Store,Cosmetics Shop,Department Store
19,East Toronto,2.0,Pub,Gastropub,Health Food Store,Pizza Place,Bakery,Cheese Shop,Accessories Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
20,Downtown Toronto,2.0,Sandwich Place,Coffee Shop,Cocktail Bar,Hotel,Beer Bar,Café,Bakery,Japanese Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant
24,Downtown Toronto,2.0,Coffee Shop,Sandwich Place,Japanese Restaurant,Sushi Restaurant,Pizza Place,Italian Restaurant,Café,Middle Eastern Restaurant,Video Game Store,Bubble Tea Shop
25,Downtown Toronto,2.0,Café,Grocery Store,Coffee Shop,Park,Athletics & Sports,Baby Store,Accessories Store,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant
30,Downtown Toronto,2.0,Coffee Shop,Café,Sandwich Place,Hotel,Gym,Asian Restaurant,Sushi Restaurant,Restaurant,Deli / Bodega,Japanese Restaurant
31,West Toronto,2.0,Park,Bakery,Pharmacy,Bank,Middle Eastern Restaurant,Bus Line,Furniture / Home Store,Brazilian Restaurant,Liquor Store,Smoke Shop
37,West Toronto,2.0,Bar,Cocktail Bar,Coffee Shop,Asian Restaurant,Vietnamese Restaurant,New American Restaurant,Vegetarian / Vegan Restaurant,Yoga Studio,Diner,Seafood Restaurant


And cluster 4

In [27]:
toronto_city_merged.loc[toronto_city_merged['Cluster Labels'] == 3, toronto_city_merged.columns[[1] + list(range(5,toronto_city_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,West Toronto,3.0,Residential Building (Apartment / Condo),Music Venue,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant
